In [3]:
import geopandas as gpd
# import folium
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles, GeoJSON
from ipywidgets import Output, SelectionSlider, VBox, HTML as HTML_widgets, Checkbox, Dropdown
from branca.colormap import linear
from pathlib import Path
#from IPython.display import display, HTML
import pandas as pd
import numpy as np
import pyproj
from math import sqrt


In [4]:
# path_project = Path("/home/ferryap/Documents/Apolline/Project_with_Mattia/map_interactive/")
path_project = Path().absolute()

In [7]:
#read the results
path_save_merged = path_project / "Europe_IGNF_grid_50km_intersection_merged_results_fine_sky_all_albedo_merged_centroid_tiles.shp"
merged_tiles_centroid = gpd.read_file(str(path_save_merged))

path_gdf_part = path_project /"Europe_IGNF_grid_50km_intersection_merged_results_fine_sky_all_albedo_BR09_1.shp"
gdf = gpd.read_file(path_gdf_part)
for i in range(2,5) : 
    path_gdf_part = path_project /f"Europe_IGNF_grid_50km_intersection_merged_results_fine_sky_all_albedo_BR09_{i}.shp"
    gdf_part = gpd.read_file(path_gdf_part)
    gdf = pd.concat([gdf,gdf_part])

if gdf.crs != 'EPSG:4326':
    gdf = gdf.to_crs('EPSG:4326')
gdf_temp = gdf[gdf['GID_0'] == "FRA"]

Index(['id', 'row_index', 'col_index', 'GID_0', 'COUNTRY', 'longitude',
       'latitude', 'tz', 'rd_lat', 'rd_long',
       ...
       'Il_05_05', 'GI_05_05', 'Al_07_05', 'Az_07_05', 'GCR_07_05', 'BT_07_05',
       'Ib_07_05', 'Il_07_05', 'GI_07_05', 'geometry'],
      dtype='object', length=258)

In [10]:
path_gdf_part = path_project /"Europe_IGNF_grid_50km_intersection_merged_results_fine_sky_all_albedo_BR09_total.shp"
gdf.to_file(str(path_gdf_part))

In [4]:
nb_tiles_y = 90 #only for 50kmx50km grid 

In [5]:
#initialization of the map and centered at the center of Europe
path_centroid_Europe = path_project / "Europe_IGNF_extent_centroid_coordinates.shp"
centroid_Europe = gpd.read_file(str(path_centroid_Europe))
center_Europe = [centroid_Europe.loc[0,'latitude'], centroid_Europe.loc[0,'longitude']]
zoom_level = 4
m = Map(center=center_Europe, zoom=zoom_level, basemap=basemap_to_tiles(basemaps.CartoDB.Positron))

In [6]:
# initialization of variable
global_coords = None
checkbox_grid_bool = False
checkbox_tile_bool = False
bool_already_one_layer = False
bool_already_two_layer = False
gdf_temp_bis = None
best_tilt_interpolation = None
global_country = "France"
gcr_value = 0.2
albedo_value = 0.2
id = None

In [7]:
#create the colormap#
vmin = 90 
vmax = 0
for a in ['01', '02', '03', '04', '05'] : 
    for g in ['005', '01', '02', '03', '04', '05', '07']: 
        column_BT = f"BT_{g}_{a}"
        vmin_temp = gdf[column_BT].min()
        vmax_temp = gdf[column_BT].max()

        if vmin_temp < vmin : 
            vmin = vmin_temp
        if vmax_temp > vmax : 
            vmax = vmax_temp
colormap = linear.YlOrRd_04.scale(vmin,vmax)

In [8]:
def update_geojson_and_colorbar():
    global geo_json_layer, geo_json_layer_bis
    if checkbox_grid_bool == False and checkbox_tile_bool ==  False and bool_already_one_layer== True : 
        m.remove_layer(geo_json_layer)

    if bool_already_two_layer == True:  
        m.remove_layer(geo_json_layer_bis)

    if checkbox_grid_bool == True and checkbox_tile_bool == True: 
        if bool_already_one_layer== True : 
            m.remove_layer(geo_json_layer)
            
        if gcr_value != 0.05 :
            column = f"BT_0{int(gcr_value * 10)}_0{int(albedo_value*10)}"
        else : 
            column = f"BT_005_0{int(albedo_value*10)}"

            if column not in gdf.columns:
                print(f"Column {column} not found in the data.")
        
        def style_callback(feature):
            value = feature['properties'][column]
            return {
                'color': 'black',
                'opacity': 1.0,
                'weight': 1,
                'fillColor': colormap(value),
                'fillOpacity': 0.99,
            }

        geo_json_data = gdf_temp.__geo_interface__
        geo_json_layer = GeoJSON(
            data=geo_json_data,
            style_callback=style_callback,
            name='GeoPandas Layer'
        )
        m.add_layer(geo_json_layer)

        if id in gdf_temp['id'] : 
            def style_callback_bis(feature):
                value = feature['properties'][column]
                return {
                    'color': 'blue',
                    'opacity': 1.0,
                    'weight': 3,
                    'fillColor': colormap(value),
                    'fillOpacity': 0,
                }
        else : 
            def style_callback_bis(feature):
                value = feature['properties'][column]
                return {
                    'color': 'blue',
                    'opacity': 1.0,
                    'weight': 3,
                    'fillColor': colormap(value),
                    'fillOpacity': 0.99,
                }

        geo_json_data_bis = gdf_temp_bis.__geo_interface__
        geo_json_layer_bis = GeoJSON(
            data=geo_json_data_bis,
            style_callback=style_callback_bis,
            name='GeoPandas Layer'
        )
        m.add_layer(geo_json_layer_bis)

    elif checkbox_grid_bool == True or checkbox_tile_bool == True: 
        if bool_already_one_layer== True : 
            m.remove_layer(geo_json_layer)
        if gcr_value != 0.05 :
            column = f"BT_0{int(gcr_value * 10)}_0{int(albedo_value*10)}"
        else : 
            column = f"BT_005_0{int(albedo_value*10)}"

        if column not in gdf.columns:
            print(f"Column {column} not found in the data.")   

        def style_callback(feature):
            value = feature['properties'][column]
            return {
                'color': 'black',
                'opacity': 1.0,
                'weight': 1,
                'fillColor': colormap(value),
                'fillOpacity': 0.99,
            }

        geo_json_data = gdf_temp.__geo_interface__
        geo_json_layer = GeoJSON(
            data=geo_json_data,
            style_callback=style_callback,
            name='GeoPandas Layer'
        )
        m.add_layer(geo_json_layer)

In [9]:
message= Output()
def display_initial_message():
    message.clear_output()
    with message:
        print(f'Click on a location on a map to know the best tilt for bifacial modules for an albedo value of {albedo_value} and Ground Coverage Ratio of {gcr_value}')
display_initial_message()

In [10]:
slider_options = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7]
slider_GCR = SelectionSlider(
    options=slider_options,
    value=0.2, 
    description='GCR:', 
    continuous_update=False, 
    orientation='horizontal', 
    readout=True 
)
def on_slider_change_GCR(change):
    global gcr_value
    gcr_value = slider_GCR.value
    display_initial_message()
    update_geojson_and_colorbar()
    if global_coords:
        handle_interaction(type='click', coordinates=global_coords)
slider_GCR.observe(on_slider_change_GCR, names='value')

In [11]:
slider_options_albedo = [0.1, 0.2, 0.3, 0.4, 0.5]
slider_albedo = SelectionSlider(
    options=slider_options_albedo,
    value=0.2,
    description='Albedo:', 
    continuous_update=False, 
    orientation='horizontal',  
    readout=True  
)

def on_slider_change_albedo(change):
    global albedo_value
    albedo_value = slider_albedo.value
    display_initial_message()
    update_geojson_and_colorbar()
    if global_coords:
        handle_interaction(type='click', coordinates=global_coords)
slider_albedo.observe(on_slider_change_albedo, names='value')

In [12]:
def create_colorbar(colormap, vmin, vmax, width=300, height=30, spacing=10, tick_height=5, legend ="Legende de la colorbar"):
    num_ticks = 10 
    tick_positions = np.linspace(vmin, vmax, num=num_ticks, endpoint=True)
    offset = +10 

    gradient = ''.join([f'<span style="background-color:{colormap(i)};width:1px;height:{height}px;display:inline-block;"></span>'
                       for i in np.linspace(vmin, vmax, num=width)])
    
    ticks = ''.join([f'<span style="position: absolute; left: {idx * (width / (num_ticks - 1))+offset}px; height: {tick_height}px; width: 1px; background-color: black; display: inline-block;"></span>'
                     for idx in range(num_ticks)])
    labels = ''.join([f'<span style="position: absolute; left: {idx * (width / (num_ticks - 1))+offset}px; text-align: center; transform: translateX(-50%);">{int(value)}</span>'
                      for idx, value in enumerate(tick_positions)])

    total_height = height + spacing + 20 + tick_height 


    return HTML_widgets(f'<div style="position: relative; width:{width}px;height:{total_height + 20}px;">'
                f'<div style="position: absolute; top: 0; left: {offset}px; width: {width}px; text-align: center;">{legend}</div>'
                f'<div style="position: absolute; top: 20px; left: {offset}px; width: {width}px; height: {height}px;">{gradient}</div>'
                f'<div style="position: absolute; top: {height + 20}px; left: 0; width: {width}px; height: {tick_height}px;">{ticks}</div>'
                f'<div style="position: absolute; top: {height + tick_height + spacing + 20}px; left: 0; width: {width}px; height: 20px; display: flex; justify-content: space-between;">{labels}</div>'
                f'</div>')

In [13]:
def transform_coordinates(lat, lon, target_proj='IGNF:ETRS89LAEA'):
    wgs84 = pyproj.CRS('EPSG:4326')
    target = pyproj.CRS(target_proj)
    transformer = pyproj.Transformer.from_crs(wgs84, target, always_xy=True)
    x, y = transformer.transform(lon, lat)
    return x, y

In [14]:
def find_id_tiles(x, y): 
    id = 0
    for i in range(0, len(merged_tiles_centroid)) : 
        top =  merged_tiles_centroid.loc[i, 'top']
        left = merged_tiles_centroid.loc[i, 'left']
        right = merged_tiles_centroid.loc[i, 'right']
        bottom = merged_tiles_centroid.loc[i, 'bottom']
        if x > left and x < right and y< top and y > bottom : 
            id = merged_tiles_centroid.loc[i, "id"]
            break
    return id

In [15]:
def find_quarter(id, coords) : 
    index = merged_tiles_centroid[merged_tiles_centroid['id']==id].index
    latitude = merged_tiles_centroid.loc[index, 'latitude']
    longitude = merged_tiles_centroid.loc[index, 'longitude']
    x_center, y_center = transform_coordinates(latitude, longitude, target_proj='IGNF:ETRS89LAEA')

    lat_point = coords[0]
    long_point = coords[1]
    x_point, y_point = transform_coordinates(lat_point, long_point, target_proj='IGNF:ETRS89LAEA')

    top = False
    right = False
    if y_point >  y_center : 
        top = True

    if x_point > x_center : 
        right = True
    
    position = ""
    if top== True and right == True : 
        position = "top right"
    if top == True and right == False : 
        position = "top left"
    if top == False and right == True : 
        position = "bottom right"
    if top == False and right == False: 
        position = "bottom left"

    return index, top, right, position, latitude, x_center

In [16]:
def find_id_voisin(id, position, nb_tiles_y):
    list_id_voisin = []
    list_id_voisin.append(id)
    if position == "top right": 
        list_id_voisin.append(id+nb_tiles_y)
        list_id_voisin.append(id+nb_tiles_y-1)
        list_id_voisin.append(id-1)
    if position == "top left": 
        list_id_voisin.append(id-nb_tiles_y)
        list_id_voisin.append(id-nb_tiles_y-1)
        list_id_voisin.append(id-1) 
    if position == "bottom right": 
        list_id_voisin.append(id+nb_tiles_y)
        list_id_voisin.append(id+nb_tiles_y+1)
        list_id_voisin.append(id+1)
    if position == "bottom left": 
        list_id_voisin.append(id-nb_tiles_y)
        list_id_voisin.append(id-nb_tiles_y+1)
        list_id_voisin.append(id+1) 

    return list_id_voisin

In [17]:
def distance_IGNF(x_1, y_1, x_2, y_2): 
    distance = sqrt((x_1-x_2)**2+(y_1-y_2)**2)
    return distance

In [18]:
def obtain_list_distance(list_id_voisin,x_point, y_point, GCR = "02", ALBEDO = "02"):
    list_distance =[]
    list_index = []
    list_best_tilt = []
    for id in list_id_voisin :
        filtered_df = merged_tiles_centroid[merged_tiles_centroid['id'] == id]
        if not filtered_df.empty:
            index_temp = filtered_df.index[0]
            list_index.append(index_temp)
            latitude_temp = filtered_df.loc[index_temp, 'latitude']
            longitude_temp = filtered_df.loc[index_temp, 'longitude']
            x_temp, y_temp =  transform_coordinates(latitude_temp, longitude_temp, target_proj='IGNF:ETRS89LAEA')
            distance_temp = distance_IGNF(x_point, y_point, x_temp, y_temp)
            list_distance.append(distance_temp)
            list_best_tilt.append(filtered_df.loc[index_temp, f'BT_{GCR}_{ALBEDO}'])
        else : 
            list_distance.append(np.nan)
            list_index.append(np.nan)
            list_best_tilt.append(np.nan)
    return list_distance, list_index, list_best_tilt

In [19]:
def interpolation_best_tilt(list_best_tilt, list_distance):
    sum_distance = 0
    sum_distance_best_tilt = 0
    i = 0
    nb_no_value = 0
    for distance in list_distance : 
        if str(distance) != "nan" and str(list_best_tilt[i])!="nan": 
            sum_distance = sum_distance + 1/(distance/1000)
            sum_distance_best_tilt = sum_distance_best_tilt + (1/(distance/1000))*list_best_tilt[i]
            nb_no_value = nb_no_value +1
        i = i +1    
    
    if nb_no_value == 0 : 
        best_tilt_interpolation = np.nan
    else : 
        best_tilt_interpolation = sum_distance_best_tilt/sum_distance

    return best_tilt_interpolation

In [20]:
coord_output = Output()
markers = []
@coord_output.capture()
def handle_interaction(**kwargs):
    global global_coords, albedo_value, id, gdf_temp, bool_already_one_layer, gdf_temp_bis, best_tilt_interpolation, coord_output, bool_already_two_layer
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        global_coords = coords
        gcr_value = slider_GCR.value
        albedo_value = slider_albedo.value
        
        if gcr_value != 0.05:
            GCR = f"0{int(gcr_value * 10)}"
        else: 
            GCR = "005"

        ALBEDO = f'0{int(albedo_value*10)}'
        
        x, y = transform_coordinates(coords[0], coords[1])
        id = find_id_tiles(x, y)
        index, top, right, position, latitude, x_center = find_quarter(id, coords)
        list_id_voisin = find_id_voisin(id, position, nb_tiles_y)
        list_distance, list_index, list_best_tilt = obtain_list_distance(list_id_voisin, x, y, GCR, ALBEDO)
        best_tilt_interpolation = interpolation_best_tilt(list_best_tilt, list_distance)
        coord_output.clear_output()
        with coord_output:
            print(f'Coordinates clicked: [{coords[0]:.3f},{coords[1]:.3f}]')
            print(f'Best tilt of the tile: {list_best_tilt[0]}')
            print(f'Best tilt interpolated from 4 nearest points : {best_tilt_interpolation:.3f}')

        if checkbox_tile_bool == True and checkbox_grid_bool == False : 
            if str(best_tilt_interpolation) != 'nan': 
                gdf_temp= gdf[gdf['id'] == id]
                update_geojson_and_colorbar()
                bool_already_one_layer = True
            else :
                coord_output.clear_output()
                with coord_output:
                    print('Location not available')

        if checkbox_tile_bool== True and checkbox_grid_bool == True : 
            if str(best_tilt_interpolation) != 'nan': 
                gdf_temp_bis= gdf[gdf['id'] == id]
                update_geojson_and_colorbar()
                bool_already_two_layer = True 
                bool_already_one_layer = True
            else :
                coord_output.clear_output()
                with coord_output:
                    print('Location not available')

        for marker in markers:
            m.remove_layer(marker)
        markers.clear()

        marker = Marker(location=coords)
        m.add_layer(marker)
        markers.append(marker)

m.on_interaction(handle_interaction)

In [21]:
show_marker_checkbox_grid = Checkbox(
    value=False,
    description='Display grid', 
    disabled=False  
)
check_box_1_output = Output()

In [22]:
show_marker_checkbox_tile = Checkbox(
    value=False, 
    description='Display tile',
    disabled=False 
)
check_box_2_output = Output()

In [23]:
colorbar = create_colorbar(colormap, vmin=vmin, vmax=vmax,width=500, height=50, spacing=5, legend = "Best tilt (°)")
colorbar.layout.visibility = 'hidden'

In [24]:
def on_checkbox_change_grid(change):
    global gdf_temp, checkbox_grid_bool,bool_already_one_layer, colorbar, gdf_temp_bis, bool_already_two_layer
    with check_box_1_output:
        check_box_1_output.clear_output()
        if checkbox_tile_bool== False: 
            if change['new']:
                if global_country == 'Europe' : 
                    gdf_temp= gdf
                else:
                    gdf_temp= gdf[gdf['COUNTRY'] == global_country]
                checkbox_grid_bool = True
                update_geojson_and_colorbar()
                bool_already_one_layer = True
            else:
                checkbox_grid_bool = False
                update_geojson_and_colorbar()
                bool_already_one_layer = False
        else : 
            if change['new']:
                checkbox_grid_bool = True
                gdf_temp_bis = gdf_temp
                
                if global_country == 'Europe' : 
                    gdf_temp= gdf
                else:
                    gdf_temp= gdf[gdf['COUNTRY'] == global_country]
                update_geojson_and_colorbar()
                bool_already_one_layer = True
                bool_already_two_layer = True

            else:
                gdf_temp = gdf_temp_bis
                gdf_temp_bis = None
                checkbox_grid_bool = False
                update_geojson_and_colorbar()
                bool_already_one_layer = True
                bool_already_two_layer = False

        if checkbox_tile_bool == True or checkbox_grid_bool == True : 
            colorbar.layout.visibility = 'visible'
        else :
            colorbar.layout.visibility = 'hidden'
show_marker_checkbox_grid.observe(on_checkbox_change_grid, names='value')

In [25]:
def on_checkbox_change_tile(change):
    global checkbox_tile_bool, colorbar, gdf_temp, bool_already_one_layer, gdf_temp_bis, bool_already_two_layer
    with check_box_2_output:
        check_box_2_output.clear_output()
        if checkbox_grid_bool == False : 
            if change['new']:
                if str(best_tilt_interpolation) != "nan" :
                    checkbox_tile_bool = True
                    gdf_temp= gdf[gdf['id'] == id]
                    update_geojson_and_colorbar()
                    bool_already_one_layer = True
                else :
                    coord_output.clear_output()
                    with coord_output:
                        print('Location not available')
                
            else:
                checkbox_tile_bool = False
                update_geojson_and_colorbar()
                bool_already_one_layer = False

        else : 
            if change['new']:
                if str(best_tilt_interpolation) != 'nan' : 
                    checkbox_tile_bool = True
                    gdf_temp_bis = gdf[gdf['id'] == id]
                    update_geojson_and_colorbar()
                    bool_already_one_layer = True
                    bool_already_two_layer = True
                else :
                    coord_output.clear_output()
                    with coord_output:
                        print('Location not available')
            else:
                gdf_temp_bis = None               
                checkbox_tile_bool = False
                update_geojson_and_colorbar()
                bool_already_one_layer = True
                bool_already_two_layer = False

        if checkbox_tile_bool == True or checkbox_grid_bool == True : 
            colorbar.layout.visibility = 'visible'
        else :
            colorbar.layout.visibility = 'hidden'

show_marker_checkbox_tile.observe(on_checkbox_change_tile, names='value')

In [26]:
options = gdf['COUNTRY'].unique().tolist()
options.sort()
options.append('Europe')
dropdown = Dropdown(
    options=options,
    value='France', 
    description='Select a country to display:'
)
output_dropdown = Output()

In [27]:
def on_dropdown_change(change):
    global global_country, gdf_temp
    with output_dropdown:
        output_dropdown.clear_output()
        print(f"Selected country : {change['new']}")
        global_country = change['new']
        if global_country == 'Europe' : 
            gdf_temp= gdf
        else:
            gdf_temp= gdf[gdf['COUNTRY'] == global_country]
        update_geojson_and_colorbar()
dropdown.observe(on_dropdown_change, names='value')

In [28]:
VBox([m,message, coord_output, colorbar,slider_GCR,slider_albedo,show_marker_checkbox_grid, show_marker_checkbox_tile, dropdown]) 